In [6]:
%run ../paths.py
CONST = CONST()

In [7]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()
import ipywidgets
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics


from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import csr_matrix, hstack


In [8]:
train = pd.read_csv(CONST.CLEANED_TRAIN)

In [9]:
train[train.columns[2:]].head()

,tweet,class,neg,pos,neu,compound,num_exclamation_marks,num_question_marks,num_symbols,stopword_count,...,num_unique_words,words_vs_unique,noun,pron,verb,adj,adv,allcaps,hashtag,user
0,thats yo bitch she my bitch too,1,0.521,0.000,0.479,-0.8225,0,0,0,3,...,7,0.875000,5,2,1,0,1,1,0,1
1,as always eli has the stupid hick hayseed look...,1,0.299,0.000,0.701,-0.8442,0,0,1,5,...,21,1.000000,8,2,5,1,1,0,0,0
2,protestors rally to support teen facing life s...,1,0.099,0.172,0.730,0.1779,0,0,1,5,...,18,0.947368,12,0,2,1,1,1,0,1
3,hoes need to stay in hoes place for real,1,0.000,0.000,1.000,0.0000,0,0,0,2,...,8,0.888889,3,0,2,1,0,0,0,0
4,i almost ran this fine red bitch over,1,0.331,0.146,0.523,-0.4818,0,0,0,2,...,8,1.000000,1,1,1,2,1,0,0,0


In [10]:
test = pd.read_csv(CONST.CLEANED_TEST)

In [6]:
test[test.columns[1:]].head()

,tweet,class,neg,pos,neu,compound,num_exclamation_marks,num_question_marks,num_symbols,stopword_count,...,num_unique_words,words_vs_unique,noun,pron,verb,adj,adv,allcaps,hashtag,user
0,what i look like ho i look like yes and you lo...,1.0,0.000,0.505,0.495,0.8814,1.0,1.0,0.0,8.0,...,10.0,0.666667,3,4,3,0,0,1,0,1
1,user lots of and from some of my favorite it i...,0.0,0.000,0.223,0.777,0.5106,0.0,0.0,0.0,7.0,...,13.0,0.928571,5,3,0,2,2,0,0,1
2,what is wrong with ppl nowadays i was conducti...,0.0,0.119,0.000,0.881,-0.4767,0.0,1.0,1.0,8.0,...,24.0,0.960000,8,0,6,4,0,0,0,0
3,user lakers are trash,1.0,0.000,0.000,1.000,0.0000,0.0,0.0,0.0,1.0,...,4.0,1.000000,2,0,1,1,0,0,0,1
4,will be playing rf simpson in at user in nov,0.0,0.000,0.160,0.840,0.2714,1.0,0.0,0.0,4.0,...,9.0,0.900000,5,0,2,1,0,0,0,1


In [7]:
#df.dropna(inplace=True)

In [11]:

drop_rows = train[train['total_length']<3]
train.drop(drop_rows.index, axis=0,inplace=True)


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42476 entries, 0 to 42477
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             42476 non-null  int64  
 1   Unnamed: 0.1           42476 non-null  int64  
 2   tweet                  42476 non-null  object 
 3   class                  42476 non-null  int64  
 4   neg                    42476 non-null  float64
 5   pos                    42476 non-null  float64
 6   neu                    42476 non-null  float64
 7   compound               42476 non-null  float64
 8   num_exclamation_marks  42476 non-null  int64  
 9   num_question_marks     42476 non-null  int64  
 10  num_symbols            42476 non-null  int64  
 11  stopword_count         42476 non-null  int64  
 12  emoji_count            42476 non-null  int64  
 13  total_length           42476 non-null  int64  
 14  num_words              42476 non-null  int64  
 15  nu

# tweets as features and only wordvectorizer

In [12]:
%run ../src/features/vectorization


In [13]:
word_vectorizer,word_selector,train_word_vec,test_word_vec = ngram_vectorize(TfidfVectorizer,train["tweet"],train["class"],test["tweet"],top_k=25000,NGRAM_RANGE=(1,2))

In [14]:
char_vectorizer,char_selector,train_char_vec,test_char_vec = ngram_vectorize(TfidfVectorizer,train["tweet"],train["class"],test["tweet"],top_k=30000,TOKEN_MODE='char',NGRAM_RANGE=(2,6))

In [15]:
train_vec_combined = hstack((train_word_vec,train_char_vec)).tocsr()

In [16]:
test_vec_combined = hstack((test_word_vec,test_char_vec)).tocsr()

In [17]:
train_ft = train[['neg','compound','allcaps','total_length','num_unique_words','adj']]
test_ft = test[['neg','compound','allcaps','total_length','num_unique_words','adj']]

In [18]:
train_ft_vec = hstack((train_vec_combined,train_ft)).tocsr()
test_ft_vec = hstack((test_vec_combined,test_ft)).tocsr()

<42476x55006 sparse matrix of type '<class 'numpy.float64'>'
	with 6926916 stored elements in Compressed Sparse Row format>

In [17]:
penalty = ['l1', 'l2']
C = [ 0.01, 0.1, 1, 10, 100 ]
class_weight = [{1:0.5, 0:0.5}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}]
solver = ['liblinear', 'saga']

param_grid = dict(penalty=penalty,
                  C=C,
                  class_weight=class_weight,
                  solver=solver)

grid_logistic = GridSearchCV(estimator=LogisticRegression(),
                    param_grid=param_grid,
                    scoring='f1',
                    verbose=2,cv=5,refit=True,n_jobs=-1)
grid_logistic_result = grid_logistic.fit(train_word_vec, train["class"])

print('Best Score: ', grid_logistic_result.best_score_)
print('Best Params: ', grid_logistic_result.best_params_)


Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear 


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear, total=   0.2s
[CV] C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear, total=   0.2s
[CV] C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear 
[CV]  C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear, total=   0.2s
[CV] C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear 
[CV]  C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear, total=   0.2s
[CV] C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear 
[CV]  C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=saga ..
[CV]  C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=saga, total=   0.6s
[CV] C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=saga ..
[CV]  C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=saga, total=   0.3s
[CV] C=0.01, class_weight={1: 0.5, 0: 0.5}, penalty=l1, solver=saga ..
[CV]  C=0.01, class_weight={1: 0.5, 0: 0.

...

[CV]  C=1, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga, total=   0.4s
[CV] C=1, class_weight={1: 0.7, 0: 0.3}, penalty=l1, solver=liblinear 
[CV]  C=1, class_weight={1: 0.7, 0: 0.3}, penalty=l1, solver=liblinear, total=   0.2s
[CV] C=1, class_weight={1: 0.7, 0: 0.3}, penalty=l1, solver=liblinear 
[CV]  C=1, class_weight={1: 0.7, 0: 0.3}, penalty=l1, solver=liblinear, total=   0.2s
[CV] C=1, class_weight={1: 0.7, 0: 0.3}, penalty=l1, solver=liblinear 
[CV]  C=1, class_weight={1: 0.7, 0: 0.3}, penalty=l1, solver=liblinear, total=   0.2s
[CV] C=1, class_weight={1: 0.7, 0: 0.3}, penalty=l1, solver=liblinear 
[CV]  C=1, class_weight={1: 0.7, 0: 0.3}, penalty=l1, solver=liblinear, total=   0.2s
[CV] C=1, class_weight={1: 0.7, 0: 0.3}, penalty=l1, solver=liblinear 
[CV]  C=1, class_weight={1: 0.7, 0: 0.3}, penalty=l1, solver=liblinear, total=   0.2s
[CV] C=1, class_weight={1: 0.7, 0: 0.3}, penalty=l1, solver=saga .....
[CV]  C=1, class_weight={1: 0.7, 0: 0.3}, penalty=l1, solver=sa

[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   0.3s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear 
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   0.3s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear 
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   0.3s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga ....
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga, total=   1.1s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga ....
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga, total=   1.1s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga ....
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga, total=   1.0s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga ....
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=sag

[CV]  C=100, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga, total= 7.5min
[CV] C=100, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga ...
[CV]  C=100, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga, total= 7.2min
[CV] C=100, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga ...
[CV]  C=100, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga, total= 7.1min
[CV] C=100, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga ...
[CV]  C=100, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga, total= 7.1min
[CV] C=100, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga ...
[CV]  C=100, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga, total= 7.1min
[CV] C=100, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear 
[CV]  C=100, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   0.7s
[CV] C=100, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear 
[CV]  C=100, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=libl

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 127.2min finished


Best Score:  0.9239517496992828
Best Params:  {'C': 10, 'class_weight': {1: 0.6, 0: 0.4}, 'penalty': 'l1', 'solver': 'saga'}


In [ ]:
model = LogisticRegression().fit(train_word_vec, train["class"])

In [23]:
#increses from 92.13 to 92.86
y = model.predict(test_word_vec)
metrics.f1_score(test['class'],y)

0.9213506933418015

In [32]:
y = grid_logistic_result.predict(train_word_vec)
print("training f1 ",metrics.f1_score(train['class'],y)*100)
print("average f1 by cv 92.86056813756783")
y = grid_logistic_result.predict(test_word_vec)
print("testing f1",metrics.f1_score(test['class'],y)*100)


training f1  96.77105398751932
average f1 by cv 92.86056813756783
testing f1 93.22643642717648


In [7]:
import pickle

pkl_filename = "../model/word_vectorizer.pkl"  


In [ ]:
%run

In [4]:
with open(pkl_filename, 'rb') as file:  
    LR = pickle.load(file)

In [8]:
with open(pkl_filename, 'rb') as file:  
    vec = pickle.load(file)

In [6]:
with open(pkl_filename, 'rb') as file:  
    selector = pickle.load(file)

In [9]:
# y = LR.predict(selector.transform(vec.transform(test["tweet"])).astype('float32'))
# metrics.f1_score(test['class'],y)*100

In [1]:
#LR.predict(selector.transform(vec.transform(["black people lives matter too"]).astype('float32')))[0]

In [35]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS


In [26]:
#sentiment_analyzer = VS()

In [2]:
#sentiment_analyzer.polarity_scores(" female  dogs +are   called  bitches")

# LR with combined vectors

In [28]:
penalty = ['l1', 'l2']
C = [0.01, 0.1, 1, 10]
class_weight = [{1:0.5, 0:0.5}, {1:0.6, 0:0.4}]
solver = ['liblinear', 'saga']

param_grid = dict(penalty=penalty,
                  C=C,
                  class_weight=class_weight,
                  solver=solver)

grid_logistic_combined = GridSearchCV(estimator=LogisticRegression(),
                    param_grid=param_grid,
                    scoring='f1',
                    verbose=1,cv=5,refit=True,n_jobs=-1)
grid_logistic_combined_result = grid_logistic_combined.fit(train_vec_combined, train["class"])

print('Best Score: ', grid_logistic_combined_result.best_score_)
print('Best Params: ', grid_logistic_combined_result.best_params_)


Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed: 2726.6min finished


Best Score:  0.9274558679886628
Best Params:  {'C': 10, 'class_weight': {1: 0.6, 0: 0.4}, 'penalty': 'l2', 'solver': 'saga'}


In [36]:
y = grid_logistic_combined_result.predict(train_vec_combined)
print("training f1 ",metrics.f1_score(train['class'],y)*100)
print("average f1 by cv 93.11986799197678")
y = grid_logistic_combined_result.predict(test_vec_combined)
print("testing f1",metrics.f1_score(test['class'],y)*100)


training f1  97.54650233938149
average f1 by cv 93.11986799197678
testing f1 93.3920704845815


In [ ]:
lrf = LogisticRegression

# LR with other features

In [33]:
penalty = ['l1', 'l2']
C = [0.1, 1, 10]
class_weight = [ {1:0.6, 0:0.4}]
solver = ['liblinear', 'saga']

param_grid = dict(penalty=penalty,
                  C=C,
                  class_weight=class_weight,
                  solver=solver)

grid_logistic_combined = GridSearchCV(estimator=LogisticRegression(),
                    param_grid=param_grid,
                    scoring='f1',
                    verbose=2,cv=5,refit=True,n_jobs=-1)
grid_logistic_ft = grid_logistic_combined.fit(train_ft_vec, train["class"])

print('Best Score: ', grid_logistic_ft.best_score_)
print('Best Params: ', grid_logistic_ft.best_params_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=liblinear 


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=liblinear, total=   2.2s
[CV] C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=liblinear 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


[CV]  C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=liblinear, total=   2.2s
[CV] C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=liblinear 
[CV]  C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=liblinear, total=   1.7s
[CV] C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=liblinear 
[CV]  C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=liblinear, total=   2.0s
[CV] C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=liblinear 
[CV]  C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=liblinear, total=   1.7s
[CV] C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga ...
[CV]  C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga, total=  14.6s
[CV] C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga ...
[CV]  C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga, total=  15.3s
[CV] C=0.1, class_weight={1: 0.6, 0: 0.4}, penalty=l1, solver=saga ...
[CV]  C=0.1, class_weight={1: 0.6, 0: 0.4}, penalt

[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=liblinear, total=   4.9s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga ....
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga, total=   9.1s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga ....
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga, total=   9.0s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga ....
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga, total=   9.1s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga ....
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga, total=   9.5s
[CV] C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga ....
[CV]  C=10, class_weight={1: 0.6, 0: 0.4}, penalty=l2, solver=saga, total=   9.1s


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 116.7min finished


Best Score:  0.9272458882041892
Best Params:  {'C': 10, 'class_weight': {1: 0.6, 0: 0.4}, 'penalty': 'l2', 'solver': 'liblinear'}


In [69]:
lr = LogisticRegression(C=10,class_weight = {1:0.6, 0:0.4},solver = 'liblinear').fit(train_ft_vec,train["class"])

In [75]:
plr = lr.predict(test_ft_vec)

In [76]:
pd.DataFrame(plr).to_csv('plr.csv')

In [77]:
pd.read_csv("plr.csv")

,Unnamed: 0,0
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
11943,11943,0
11944,11944,1
11945,11945,0
11946,11946,0


In [90]:
att = pd.read_csv("/home/sandhya/Downloads/cnn.csv")

In [91]:
att['0'] = (att['0']>0.48)* 1

In [92]:
pd.DataFrame(att['0']).to_csv("a.csv")

In [93]:
import pandas as pd
import sys

first_file = 'a.csv'
second_file = 'plr.csv'

def corr(first_file, second_file):
  first_df = pd.read_csv(first_file,index_col=0)
  second_df = pd.read_csv(second_file,index_col=0)
  # assuming first column is `prediction_id` and second column is `prediction`
  prediction = first_df.columns[0]
  # correlation
  print("Finding correlation between: {} and {}".format(first_file,second_file))
  print("Column to be measured: {}".format(prediction))
  #print(" score: {}".format(first_df[prediction]))
  #print(" score1: {}".format(second_df[prediction])
  
  print("Pearson's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='pearson')))
  print("Kendall's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='kendall')))
  print("Spearman's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='spearman')))

corr(first_file, second_file)

Finding correlation between: a.csv and plr.csv
Column to be measured: 0
Pearson's correlation score: 0.6810190209268407
Kendall's correlation score: 0.6810190209268443
Spearman's correlation score: 0.6810190209268444


In [94]:
import pandas as pd
import sys

first_file = 'a.csv'
second_file = 'xgb.csv'

def corr(first_file, second_file):
  first_df = pd.read_csv(first_file,index_col=0)
  second_df = pd.read_csv(second_file,index_col=0)
  # assuming first column is `prediction_id` and second column is `prediction`
  prediction = first_df.columns[0]
  # correlation
  print("Finding correlation between: {} and {}".format(first_file,second_file))
  print("Column to be measured: {}".format(prediction))
  #print(" score: {}".format(first_df[prediction]))
  #print(" score1: {}".format(second_df[prediction])
  
  print("Pearson's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='pearson')))
  print("Kendall's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='kendall')))
  print("Spearman's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='spearman')))

corr(first_file, second_file)

Finding correlation between: a.csv and xgb.csv
Column to be measured: 0
Pearson's correlation score: 0.6873682517685868
Kendall's correlation score: 0.6873682517685873
Spearman's correlation score: 0.6873682517685872


# XGBoost(GBD)
### Ensemble
* Bagging RF(decreases Variance)
* Boosting GBM (decreases Bias)
### [Hyperparameter tuning](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)


In [20]:
from xgboost import XGBClassifier

In [29]:
xgb1 = {
 'learning_rate' : [1],
 'n_estimators' : range(40,151,10),
 'max_depth' : [5],
 'min_child_weight' : [1],
 'gamma' : [0],
 'subsample' : [0.8],
 'colsample_bytree' : [0.8],
 'objective' : ['binary:logistic'],
 'seed' : [27]}

In [30]:
xgbm1 = GridSearchCV(XGBClassifier(),xgb1,refit=True,cv=5,verbose=2,n_jobs=-1)
xgbm1.fit(train_ft_vec,train["class"])
print('Best Score: ', xgbm1.best_score_)
print('Best Params: ', xgbm1.best_params_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=40, objective=binary:logistic, seed=27, subsample=0.8 


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=40, objective=binary:logistic, seed=27, subsample=0.8, total= 1.1min
[CV] colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=40, objective=binary:logistic, seed=27, subsample=0.8 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=40, objective=binary:logistic, seed=27, subsample=0.8, total= 1.1min
[CV] colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=40, objective=binary:logistic, seed=27, subsample=0.8 
[CV]  colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=40, objective=binary:logistic, seed=27, subsample=0.8, total= 1.0min
[CV] colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=40, objective=binary:logistic, seed=27, subsample=0.8 
[CV]  colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=40, objective=binary:logistic, seed=27, subsample=0.8, total= 1.1min
[CV] colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=40, objective=binary:logistic, seed=27, subsample=0.8 
[CV]  colsample_bytree=0.8, gam

[CV]  colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=90, objective=binary:logistic, seed=27, subsample=0.8, total= 2.0min
[CV] colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=90, objective=binary:logistic, seed=27, subsample=0.8 
[CV]  colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=90, objective=binary:logistic, seed=27, subsample=0.8, total= 2.1min
[CV] colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=90, objective=binary:logistic, seed=27, subsample=0.8 
[CV]  colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=90, objective=binary:logistic, seed=27, subsample=0.8, total= 2.2min
[CV] colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=100, objective=binary:logistic, seed=27, subsample=0.8 
[CV]  colsample_bytree=0.8, ga

[CV]  colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=140, objective=binary:logistic, seed=27, subsample=0.8, total= 2.9min
[CV] colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=140, objective=binary:logistic, seed=27, subsample=0.8 
[CV]  colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=140, objective=binary:logistic, seed=27, subsample=0.8, total= 2.9min
[CV] colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=150, objective=binary:logistic, seed=27, subsample=0.8 
[CV]  colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=150, objective=binary:logistic, seed=27, subsample=0.8, total= 3.1min
[CV] colsample_bytree=0.8, gamma=0, learning_rate=1, max_depth=5, min_child_weight=1, n_estimators=150, objective=binary:logistic, seed=27, subsample=0.8 
[CV]  colsample_bytree=0.

[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 145.3min finished


Best Score:  0.9203786125437695
Best Params:  {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 1, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 50, 'objective': 'binary:logistic', 'seed': 27, 'subsample': 0.8}


In [22]:
xg = XGBClassifier().fit(train_ft_vec,train["class"])

In [23]:
predict = xg.predict(test_ft_vec)

In [24]:
pd.DataFrame(predict).to_csv('xgb.csv')


In [95]:
#pd.read_csv("xgb.csv")

<img src="https://miro.medium.com/max/1280/0*08KrYhXpVQdUXWrX" />

<img src="https://miro.medium.com/max/1280/1*6HVomcqW7BWuZ2vvGOEptw.png" />

In [19]:
param_grid_svm = {'C': [0.01,0.1,1, 10], 'gamma': [1,0.1,0.01,0.001],'kernel': ['linear']}

In [20]:
#by default 5 fold cv
grid_svm = GridSearchCV(SVC(),param_grid_svm,refit=True,cv=5,verbose=2,n_jobs=-1)
grid_svm.fit(train_word_vec,train["class"])
print('Best Score: ', grid_svm.best_score_)
print('Best Params: ', grid_svm.best_params_)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=0.01, gamma=1, kernel=linear ..................................


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... C=0.01, gamma=1, kernel=linear, total= 3.3min
[CV] C=0.01, gamma=1, kernel=linear ..................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.3min remaining:    0.0s


[CV] ................... C=0.01, gamma=1, kernel=linear, total= 3.1min
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] ................... C=0.01, gamma=1, kernel=linear, total= 3.1min
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] ................... C=0.01, gamma=1, kernel=linear, total= 3.1min
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] ................... C=0.01, gamma=1, kernel=linear, total= 3.1min
[CV] C=0.01, gamma=0.1, kernel=linear ................................
[CV] ................. C=0.01, gamma=0.1, kernel=linear, total= 3.1min
[CV] C=0.01, gamma=0.1, kernel=linear ................................
[CV] ................. C=0.01, gamma=0.1, kernel=linear, total= 3.1min
[CV] C=0.01, gamma=0.1, kernel=linear ................................
[CV] ................. C=0.01, gamma=0.1, kernel=linear, total= 3.1min
[CV] C=0.01, gamma=0.1, kernel=linear ................................
[CV] .

[CV] .................. C=1, gamma=0.001, kernel=linear, total= 4.8min
[CV] C=10, gamma=1, kernel=linear ....................................
[CV] ..................... C=10, gamma=1, kernel=linear, total= 8.1min
[CV] C=10, gamma=1, kernel=linear ....................................
[CV] ..................... C=10, gamma=1, kernel=linear, total= 6.7min
[CV] C=10, gamma=1, kernel=linear ....................................
[CV] ..................... C=10, gamma=1, kernel=linear, total= 8.3min
[CV] C=10, gamma=1, kernel=linear ....................................
[CV] ..................... C=10, gamma=1, kernel=linear, total= 7.0min
[CV] C=10, gamma=1, kernel=linear ....................................
[CV] ..................... C=10, gamma=1, kernel=linear, total= 8.4min
[CV] C=10, gamma=0.1, kernel=linear ..................................
[CV] ................... C=10, gamma=0.1, kernel=linear, total= 8.2min
[CV] C=10, gamma=0.1, kernel=linear ..................................
[CV] .

[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 351.7min finished


Best Score:  0.9399896748835307
Best Params:  {'C': 1, 'gamma': 1, 'kernel': 'linear'}


In [45]:
param_grid_svm = {'C': [1], 'gamma': [100],'kernel': ['linear']}

In [46]:
grid_svm = GridSearchCV(SVC(),param_grid_svm,refit=True,cv=5,verbose=2,n_jobs=-1)
grid_svm.fit(train_word_vec,train["class"])
print('Best Score: ', grid_svm.best_score_)
print('Best Params: ', grid_svm.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] C=1, gamma=100, kernel=linear ...................................


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................... C=1, gamma=100, kernel=linear, total= 3.9min
[CV] C=1, gamma=100, kernel=linear ...................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.9min remaining:    0.0s


[CV] .................... C=1, gamma=100, kernel=linear, total= 3.8min
[CV] C=1, gamma=100, kernel=linear ...................................
[CV] .................... C=1, gamma=100, kernel=linear, total=18.2min
[CV] C=1, gamma=100, kernel=linear ...................................
[CV] .................... C=1, gamma=100, kernel=linear, total= 4.0min
[CV] C=1, gamma=100, kernel=linear ...................................
[CV] .................... C=1, gamma=100, kernel=linear, total= 3.8min


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 33.7min finished


Best Score:  0.9399896748835307
Best Params:  {'C': 1, 'gamma': 100, 'kernel': 'linear'}


In [42]:
y = grid_svm.predict(train_word_vec)
print("training f1 ",metrics.f1_score(train['class'],y)*100)
print("average f1 by cv 94.23910454870208")
y = grid_svm.predict(test_word_vec)
print("testing f1",metrics.f1_score(test['class'],y)*100)


training f1  95.08724675020494
average f1 by cv 94.23910454870208
testing f1 93.0330015715034


In [29]:
param_grid_svm_combined = {'C': [0.01,0.1,1], 'gamma': [10,1,0.1],'kernel': ['linear']}

In [31]:
grid_svm_combined = GridSearchCV(SVC(),param_grid_svm_combined,refit=True,cv=5,verbose=2,n_jobs=-1)
grid_svm_combined.fit(train_word_vec,train["class"])
print('Best Score: ', grid_svm_combined.best_score_)
print('Best Params: ', grid_svm_combined.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] C=0.01, gamma=10, kernel=linear .................................


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................. C=0.01, gamma=10, kernel=linear, total= 3.2min
[CV] C=0.01, gamma=10, kernel=linear .................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.2min remaining:    0.0s


[CV] .................. C=0.01, gamma=10, kernel=linear, total= 3.8min
[CV] C=0.01, gamma=10, kernel=linear .................................
[CV] .................. C=0.01, gamma=10, kernel=linear, total= 4.1min
[CV] C=0.01, gamma=10, kernel=linear .................................
[CV] .................. C=0.01, gamma=10, kernel=linear, total= 4.0min
[CV] C=0.01, gamma=10, kernel=linear .................................
[CV] .................. C=0.01, gamma=10, kernel=linear, total= 4.1min
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] ................... C=0.01, gamma=1, kernel=linear, total= 4.0min
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] ................... C=0.01, gamma=1, kernel=linear, total= 4.0min
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] ................... C=0.01, gamma=1, kernel=linear, total= 4.5min
[CV] C=0.01, gamma=1, kernel=linear ..................................
[CV] .

[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 158.2min finished


Best Score:  0.9399896748835307
Best Params:  {'C': 1, 'gamma': 10, 'kernel': 'linear'}


In [334]:
pipe = Pipeline(
        [('select', SelectFromModel(LogisticRegression(class_weight='balanced',
                                                  penalty="l1", C=0.01))),
        ('model', LogisticRegression(class_weight='balanced',penalty='l2'))])

In [35]:
%run ../src/classifier/NBSVM

In [36]:
nb_svm = NbSvmClassifier(C=10)

In [531]:
metrics.f1_score(test_data['class'],predict)

0.9304790605840174

In [575]:
df3.shape

(11866, 30000)

In [576]:
from scipy.sparse import csr_matrix, hstack


In [597]:
#frame = [train_vec.toarray(),train_data[Columns]]
df2 = hstack((train_vec,train_vec_char,train_data[Columns])).tocsr()

In [19]:
#nb_svm.fit(train_vec,train["class"])

In [483]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [544]:
scores = []
classifier = LogisticRegression(C=10, solver='sag')
cv_score = np.mean(cross_val_score(classifier, df2, train_data["class"] ,cv=10, scoring='f1'))
scores.append(cv_score)
print('CV score for class {} is {}'.format(class_name, cv_score))

   

CV score for class 0 is 0.931595865109229


In [545]:
classifier.fit(df2, train_data["class"])
print('Total CV score is {}'.format(np.mean(scores)))

Total CV score is 0.931595865109229


In [570]:
model = LogisticRegression(solver='sag')
sfm = SelectFromModel(model, threshold=0.2)
train_sparse_matrix = sfm.fit_transform(df2, train_data['class'])
test_sparse_matrix = sfm.transform(df3)
d_train = lgb.Dataset(train_sparse_matrix, label=train_data["class"])
d_valid = lgb.Dataset(test_sparse_matrix, label=test_data["class"])
watchlist = [d_train, d_valid]
params = {'learning_rate': 0.2,
             'application': 'binary',
              'num_leaves': 31,
              'verbosity': -1,
              'metric': 'f1',
              'data_random_seed': 2,
              'bagging_fraction': 0.8,
              'feature_fraction': 0.6,
              'nthread': 4,
              'lambda_l1': 1,
          'lambda_l2': 1}
model1 = lgb.train(params,
                      train_set=d_train,
                      verbose_eval=10)
    


In [552]:
import lightgbm as lgb



In [557]:
?lgb.train

In [579]:
p = model.predict(test_sparse_matrix)

In [625]:
metrics.f1_score(test_data['class'],p>0.42)

0.9315718856435132

In [615]:
select = SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l1",C=10,solver='liblinear'))
X_ = select.fit_transform(df2,train_data["class"])

In [627]:
model = LinearSVC(class_weight='balanced',C=0.1, penalty='l2', loss='squared_hinge').fit(X_, train_data["class"])

In [628]:
p1 = model.predict(select.transform(df3))

In [629]:
metrics.f1_score(test_data['class'],p1)

0.9238144867118291